In [1]:
#DEPENDENCIES
import pandas as pd
import pandas_read_xml as pdx
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
url = "https://mars.nasa.gov/news/?page=0&per_page=10&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [3]:
# Retrieve page with the requests module
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [4]:
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [18]:
news_date = soup.find('div', class_='list_date').text == 'December  8, 2020'
news_date

True

In [19]:
news = soup.find_all('div', attrs={'class':'list_date'})
news_title = ''
for div in news:
    try:
        if div.find('div', attrs={'list_date'}) == 'December  8, 2020' :
            nextDiv = div.findNext('div', attrs={'class': 'content_title'})
            news_title= nextDiv.find('a').text
    except AttributeError:
        print('attribute missing')

print(news_title)
#getting the data div using findNext
#nextDiv =  div.findNext('div', attrs={'class':'data'})
#use findNext again on previous object to get to the anchor tag
#fileName = nextDiv.findNext('a').text

In [21]:
mars_news = soup.find('ul', class_='item_list')
news_title = mars_news.find('div', class_='content_title').text
news_title

"5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover"

In [23]:
news_p = mars_news.find('div', class_='article_teaser_body').text
news_p

"The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative."

In [25]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [26]:
full_image = browser.find_by_id('full_image')
full_image.click()

In [27]:
browser.is_element_present_by_text('more info')
more_info = browser.links.find_by_partial_text('more info')
more_info.click()

In [32]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
main_image = soup.find('img', class_='main_image')['src']
main_image
#https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19324_hires.jpg
url = f"https://www.jpl.nasa.gov{main_image}"
print(url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19324_hires.jpg


In [40]:
url = 'https://space-facts.com/mars/'
mars_facts_df = pd.read_html(url)[0]
mars_facts_df.columns=['Description', 'Facts']
mars_facts_df.set_index('Description', inplace = True)
mars_facts_df.head()


,Facts
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [11]:
#results = soup.find('a', class_='div.list_date')
news_titles = soup.find_all('div', attrs={'class':'content_title'})
for a in news_titles:
    try:
        print(a.find('a').text)
        if a.find('a').text == "5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover":
            news_title = a.find('a').text
    except AttributeError:
        print('text not found')
print(news_title)

text not found
5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover
MOXIE Could Help Future Rockets Launch Off Mars
Hear Audio From NASA's Perseverance As It Travels Through Deep Space
Mars Is Getting a New Robotic Meteorologist
Heat and Dust Help Launch Martian Water Into Space, Scientists Find
NASA's Curiosity Takes Selfie With 'Mary Anning' on the Red Planet
Independent Review Indicates NASA Prepared for Mars Sample Return Campaign
NASA's Perseverance Rover 100 Days Out
NASA's Perseverance Rover Is Midway to Mars 
Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth
NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars
NASA InSight's 'Mole' Is Out of Sight
NASA's Perseverance Rover Will Peer Beneath Mars' Surface 
AI Is Helping Scientists Discover Fresh Craters on Mars
NASA's New Mars Rover Is Ready for Space Lasers
NASA's New Mars Rover Will Use X-Rays to Hunt Fossils
My Culture, My Voice
NASA Readies Perseverance Mars Rover's Earthly Twin 
NASA Eng

In [ ]:
#news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"
news_title = soup.find('div a href', class_='content_title').text

#news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for 
#launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."